In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import interpolate, ndimage
import pyvista as pv
pv.set_jupyter_backend('trame')

## Define functions

### Generating rw

In [2]:
def sample_k(k_mean,k_cov):
    return np.random.multivariate_normal(k_mean,k_cov)

#### Superpositioning the random wave ####
def sample_wave(r_grid,k_mean,k_cov,n_wave = 100):
    rho = np.zeros_like(r_grid[0])
    r_grid = [r.astype(np.float32) for r in r_grid]
    for i in range(n_wave):
        phi = np.random.rand()*2*np.pi # random phase
        k_sample = sample_k(k_mean,k_cov)
        k_dot_r = np.sum([r_grid[x]*k_sample[x] for x in range(3)],axis=0)
        rho_i = np.cos(k_dot_r.astype(np.float32) + phi) # cos(k_n.r + phi_n)
        rho += rho_i

    rho = np.sqrt(2/n_wave)*rho
    
    return rho

# Misoientation
def rotation_matrix(axis, phi):
    """
    Return the rotation matrix associated with counterclockwise rotation about
    the given axis by theta radians.
    """
    axis = np.asarray(axis)
    axis = axis / np.sqrt(np.dot(axis, axis))
    a = np.cos(phi / 2.0)
    b, c, d = -axis * np.sin(phi / 2.0)
    aa, bb, cc, dd = a * a, b * b, c * c, d * d
    bc, ad, ac, ab, bd, cd = b * c, a * d, a * c, a * b, b * d, c * d
    return np.array([[aa + bb - cc - dd, 2 * (bc + ad), 2 * (bd - ac)],
                     [2 * (bc - ad), aa + cc - bb - dd, 2 * (cd + ab)],
                     [2 * (bd + ac), 2 * (cd - ab), aa + dd - bb - cc]])

def sample_wave_MO(r_grid, k_mean, k_cov, n_wave = 100, kappa=1e8):
    rho = np.zeros_like(r_grid[0])
    r_grid = [r.astype(np.float32) for r in r_grid]
    for i in range(n_wave):
        k_sample = sample_k(k_mean,k_cov)

        # misorientation
        """
        https://en.wikipedia.org/wiki/Von_Mises%E2%80%93Fisher_distribution
        https://doi.org/10.1080/03610919408813161
        """
        sigma = 1e-6
        xi = np.random.rand()
        theta = np.random.rand()*2*np.pi
        W = 1+1/kappa*(np.log(xi*(1-(xi-1)/xi*np.exp(-2*kappa))))
        phi = np.arccos(W)
        axis = [np.cos(theta),np.sin(theta),0]
        R = rotation_matrix(axis,phi)
        k_sample_rot = R@k_sample

        k_dot_r = np.sum([r_grid[x]*k_sample_rot[x] for x in range(3)],axis=0)
        phi_r = np.random.rand()*2*np.pi # random phase
        rho_i = np.cos(k_dot_r.astype(np.float32) + phi_r) # cos(k_n.r + phi_n)
        rho += rho_i

    rho = np.sqrt(2/n_wave)*rho
    
    return rho

def sample_wave_MO_complex(r_grid, k_mean, k_cov, n_wave = 100, kappa=1e8):
    rho = np.zeros_like(r_grid[0]).astype('complex64')
    r_grid = [r.astype(np.float32) for r in r_grid]
    for i in range(n_wave):
        k_sample = sample_k(k_mean,k_cov)

        # misorientation
        """
        https://en.wikipedia.org/wiki/Von_Mises%E2%80%93Fisher_distribution
        https://doi.org/10.1080/03610919408813161
        """
        sigma = 1e-6
        xi = np.random.rand()
        theta = np.random.rand()*2*np.pi
        W = 1+1/kappa*(np.log(xi*(1-(xi-1)/xi*np.exp(-2*kappa))))
        phi = np.arccos(W)
        axis = [np.cos(theta),np.sin(theta),0]
        R = rotation_matrix(axis,phi)
        k_sample_rot = R@k_sample

        k_dot_r = np.sum([r_grid[x]*k_sample_rot[x] for x in range(3)],axis=0)
        phi_r = np.random.rand()*2*np.pi # random phase
        rho_i = np.exp(1j*(k_dot_r + phi_r)) # cos(k_n.r + phi_n)
        rho += rho_i.astype('complex64')

    rho = np.sqrt(2/n_wave)*rho
    
    return rho

## Generate an isotropic randomwave

In [3]:
## define parameters
sigma_k = 1e-3
kappa = 1e-3
alpha = 0.0
scale = 5 # how many wavelengths per 1 length unit

k_mean = np.array([0,0,1])
k_var = np.array([0,0,sigma_k**2])
k_cov = np.diag(k_var)

## define grid
n_grid = 128
x = np.linspace(-1,1,n_grid+1)
y = np.linspace(-1,1,n_grid+1)
z = np.linspace(-1,1,n_grid+1)
r_grid = np.meshgrid(x,y,z) 

## miscellation parameters
n_wave = 32

rho = sample_wave_MO_complex(r_grid,k_mean,k_cov,n_wave = n_wave, kappa = kappa)

rho_real = np.real(rho)
rho_imag = np.imag(rho)
rho_abs = np.abs(rho)

In [5]:
## visualization
pv.set_plot_theme('document')
pl = pv.Plotter(window_size=[600, 600])
pl.enable_anti_aliasing('msaa')

grid = pv.StructuredGrid(r_grid[0], r_grid[1], r_grid[2])
grid["vol"] = rho_real.flatten('F')
mesh = grid.contour([alpha])

backface_params = dict(color='#303030',
                            ambient=0.2, diffuse=0.8, specular=0.1, specular_power=10,
                            opacity=1
                            )
pl.add_mesh(mesh, show_scalar_bar=False, color='#A0A0A0',  
            ambient=0.2, diffuse=0.8, specular=0.1, specular_power=10,
            backface_params=backface_params, 
            smooth_shading=True, 
            opacity=1
            )

if 1:
    # camera setting
    pl.enable_parallel_projection()
    pl.camera_position = 'yz'
    pl.camera.reset_clipping_range()
else:
    # camera setting
    pl.camera_position = 'yz'
    pl.camera.azimuth = -60.0
    pl.camera.elevation = 24.0
    pl.camera.reset_clipping_range()

# light setting
light = pv.Light()
light.set_direction_angle(21, -55.0)
light.attenuation_values = (0,0,2)
pl.add_light(light)

pl.add_bounding_box()
pl.show(screenshot='tmp_rw.png')

Widget(value='<iframe src="http://localhost:33191/index.html?ui=P_0x7fb760b086a0_1&reconnect=auto" class="pyvi…